**Câu 1 (2 điểm).** Xét hàm số $f(x) = (x - 2)^4 + 0.5(x + 1)^2$.

Yêu cầu:
1. Viết đạo hàm bậc nhất và đạo hàm bậc hai của $f(x)$.
2. Chạy thuật toán Newton từ nhiều điểm khởi tạo khác nhau.
3. Vẽ đồ thị 1D thể hiện các điểm lặp.


In [ ]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt


# ===============================================================
# 🧩 HÀM 1 — TÍNH ĐẠO HÀM BẬC NHẤT VÀ BẬC HAI (tổng quát)
# ===============================================================
def compute_derivatives(f_expr, x):
    """
    Trả về đạo hàm bậc nhất và bậc hai của f(x)
    """
    f_prime = sp.diff(f_expr, x)
    f_double = sp.diff(f_prime, x)
    return f_prime, f_double


# ===============================================================
# 🧩 HÀM 2 — CHẠY THUẬT TOÁN NEWTON-RAPHSON
# ===============================================================
def newton_method_1d(f, df, d2f, x0, tol=1e-8, max_iter=50):
    """
    Thực hiện Newton-Raphson cho 1 biến.
    
    Parameters
    ----------
    f, df, d2f : callable
        Hàm f(x), f'(x), f''(x)
    x0 : float
        Điểm khởi tạo
    tol : float
        Ngưỡng hội tụ
    max_iter : int
        Số vòng lặp tối đa

    Returns
    -------
    history : list các giá trị x qua các vòng lặp
    """
    x = x0
    history = [x]

    for _ in range(max_iter):
        grad = df(x)
        hess = d2f(x)
        if abs(hess) < 1e-12:
            print(f"⚠️ Hessian ≈ 0 tại x={x:.4f}, dừng.")
            break

        x_new = x - grad / hess
        history.append(x_new)
        if abs(x_new - x) < tol:
            break
        x = x_new

    return history
def newton_method(f, df, d2f, x0, tol=1e-8, max_iter=50):
    """
    Newton-Raphson linh hoạt tự xử lý 1D và nD
    đầu vào giữ nguyên yêu cầu của đề gốc.

    - Nếu x0 là số -> Newton 1D
    - Nếu x0 là vector -> Newton nD
    """
    
    # =====================================================
    # CASE 1: 1D input
    # =====================================================
    if isinstance(x0, (int, float, np.floating, np.integer)):

        x = x0
        history = [x]

        for _ in range(max_iter):
            grad = df(x)
            hess = d2f(x)

            if abs(hess) < 1e-12:
                print(f"⚠ Hessian ≈ 0 tại x={x:.4f}, dừng.")
                break

            x_new = x - grad/hess
            history.append(x_new)

            if abs(x_new - x) < tol:
                break

            x = x_new
        
        return history
    

    # =====================================================
    # CASE 2: ND input automatically detected
    # =====================================================
    else:

        x = np.array(x0, dtype=float)
        history = [x.copy()]

        for _ in range(max_iter):

            grad = np.asarray(df(*x), dtype=float)
            hess = np.asarray(d2f(*x), dtype=float)

            # Hessian inverse solve step
            try:
                step = np.linalg.solve(hess, grad)
            except np.linalg.LinAlgError:
                print("⚠ Hessian singular → fallback GD")
                step = grad

            x_new = x - step

            history.append(x_new.copy())

            if np.linalg.norm(x_new - x) < tol:
                break
            
            x = x_new
        
        return history


# ===============================================================
# 🧩 HÀM 3 — VẼ ĐỒ THỊ 1D VÀ ĐIỂM LẶP NEWTON
# ===============================================================
def plot_newton_iterations(f, results, x_range=(-4, 4)):
    """
    Vẽ đồ thị f(x) và các điểm lặp Newton từ nhiều khởi tạo.
    """
    x_vals = np.linspace(x_range[0], x_range[1], 400)
    y_vals = f(x_vals)

    plt.figure(figsize=(8, 5))
    plt.plot(x_vals, y_vals, 'b-', label='$f(x)$')

    colors = ['r', 'g', 'orange', 'purple', 'brown']
    for i, (x0, hist) in enumerate(results.items()):
        c = colors[i % len(colors)]
        plt.plot(hist, f(np.array(hist)), 'o--', color=c,
                 label=f'x0={x0} → x*={hist[-1]:.4f}')
    
    plt.xlabel('x')
    plt.ylabel('f(x)')
    plt.title('🧮 Đồ thị f(x) và các điểm lặp Newton-Raphson')
    plt.legend()
    plt.grid(True)
    plt.show()

def plot_newton(f, traj, variables, x_range=(-4,4)):
    """
    Vẽ Newton trajectory cho 1D / 2D / nD.
    Tự xử lý:
        - 1 quỹ đạo (list/ndarray)
        - nhiều quỹ đạo (dictionary)
    """
    dim = len(variables)

    # ========================================================
    # CASE: multiple runs provided (dictionary)
    # ========================================================
    if isinstance(traj, dict):    
        # lấy bất kỳ phần tử đầu tiên để đo chiều
        first_key = next(iter(traj))
        first_traj = traj[first_key]

        # ----------------- CASE 1D MULTI PATH -----------------
        if isinstance(first_traj[0], (float,int,np.floating,np.integer)) and dim == 1:

            xs = np.linspace(x_range[0], x_range[1], 400)
            plt.plot(xs, f(xs), 'b-')

            colors = ['r','g','orange','purple','brown']

            for i, (x0, path) in enumerate(traj.items()):
                c = colors[i % len(colors)]
                plt.plot(path, f(np.array(path)), "o--", color=c, 
                         label=f"x0={x0} → x*={path[-1]:.4f}")

            plt.legend()
            plt.grid(True)
            plt.show()
            return
        
        # ----------------- CASE 2D MULTI PATH -----------------
        if dim == 2:
            x1_vals = np.linspace(-4,4,200)
            x2_vals = np.linspace(-4,4,200)
            X1, X2 = np.meshgrid(x1_vals, x2_vals)

            Z = f(X1, X2)
            plt.contour(X1, X2, Z, levels=30)

            for _, path in traj.items():
                path = np.array(path)
                plt.plot(path[:,0], path[:,1], 'o--')

            plt.title("Newton trajectories (2D)")    
            plt.grid(True)
            plt.show()
            return

        # ----------------- CASE ND MULTI PATH -----------------
        else:
            print("nD (>=3) detected → plot error curves:")
            for _, path in traj.items():
                errors = [
                    np.linalg.norm(np.array(x) - np.array(path[-1])) 
                    for x in path
                ]
                plt.plot(errors)
            plt.title("Convergence curves (nD)")
            plt.grid(True)
            plt.show()
            return
    
    
    # ========================================================
    # CASE: single run (list)
    # ========================================================
    else:
        # original behavior
        return plot_newton(f, {0: traj}, variables)

# ===============================================================
# 🧩 HÀM 4 — HÀM ĐIỀU PHỐI TOÀN BỘ QUY TRÌNH (chạy hoàn chỉnh)
# ===============================================================
def run_newton_general(f_expr, x_symbol, x0_list, tol=1e-8, show_plot=True):
    """
    Chạy toàn bộ quy trình:
    - Tính đạo hàm
    - Duyệt qua nhiều điểm khởi tạo
    - Vẽ đồ thị nếu cần
    """
    # --- 1. Đạo hàm ---
    f_prime, f_double = compute_derivatives(f_expr, x_symbol)

    # --- 2. Tạo hàm lambda ---
    f = sp.lambdify(x_symbol, f_expr, 'numpy')
    df = sp.lambdify(x_symbol, f_prime, 'numpy')
    d2f = sp.lambdify(x_symbol, f_double, 'numpy')

    # --- 3. Chạy Newton cho từng điểm ---
    results = {}
    for x0 in x0_list:
        hist = newton_method(f, df, d2f, x0, tol=tol)
        results[x0] = hist

    # --- 4. Hiển thị kết quả ---
    print("Đạo hàm bậc nhất:", f_prime)
    print("Đạo hàm bậc hai :", f_double)
    for x0, hist in results.items():
        print(f"→ x0={x0:>5}: x*={hist[-1]:.6f}, số vòng lặp={len(hist)-1}")

    # --- 5. Vẽ ---
    if show_plot:
        plot_newton(f, results, [x_symbol])


# ===============================================================
# 🧮 VÍ DỤ CHẠY CHO BÀI TOÁN TRONG ĐỀ
# ===============================================================
if __name__ == "__main__":
    x = sp.Symbol('x')
    f_expr = (x - 2)**4 + 0.5*(x + 1)**2

    run_newton_general(
        f_expr=f_expr,
        x_symbol=x,
        x0_list=[-3, 0, 1, 3],
        tol=1e-8,
        show_plot=True
    )


In [ ]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt


# ===============================================================
# 1️⃣ TÍNH GRADIENT & HESSIAN CHO ND
# ===============================================================
def compute_derivatives_nd(f_expr, variables):
    grad = [sp.diff(f_expr, v) for v in variables]
    hess = sp.hessian(f_expr, variables)
    return grad, hess


# ===============================================================
# 2️⃣ NEWTON MULTIDIMENSION
# ===============================================================
def newton_method_nd(f, grad_func, hess_func, x0, tol=1e-8, max_iter=20):

    x = np.array(x0, dtype=float)
    traj = [x.copy()]

    for _ in range(max_iter):

        g = np.array(grad_func(*x), dtype=float)
        H = np.array(hess_func(*x), dtype=float)

        if np.linalg.norm(g) < tol:
            break

        try:
            step = np.linalg.solve(H, g)
        except:
            step = g  # fallback GD

        x_new = x - step
        traj.append(x_new.copy())

        if np.linalg.norm(x_new - x) < tol:
            break

        x = x_new

    return traj


# ===============================================================
# 3️⃣ PLOT 2D CONTOUR TRAJECTORY
# ===============================================================
def plot_newton_2d(f, results, xrange=(-4,4), yrange=(-4,4)):

    xs = np.linspace(xrange[0], xrange[1], 200)
    ys = np.linspace(yrange[0], yrange[1], 200)
    X, Y = np.meshgrid(xs, ys)

    Z = f(X, Y)

    plt.figure(figsize=(7,6))
    plt.contour(X, Y, Z, levels=35)
    plt.title("Newton Trajectory 2D")

    for x0, traj in results.items():
        P = np.array(traj)
        plt.plot(P[:,0], P[:,1], "o--", label=f"{x0}")

    plt.legend()
    plt.grid()
    plt.show()


# ===============================================================
# 4️⃣ HÀM ĐIỀU PHỐI CHUNG
# ===============================================================
def run_newton_general_nd(f_expr, variables, x0_list, tol=1e-8, show_plot=True):

    # 1. symbolic derivative
    grad_expr, hess_expr = compute_derivatives_nd(f_expr, variables)

    # 2. convert to numeric
    f = sp.lambdify(variables, f_expr, "numpy")
    grad_f = sp.lambdify(variables, grad_expr, "numpy")
    hess_f = sp.lambdify(variables, hess_expr, "numpy")

    # 3. run Newton for multiple starting points
    results = {}
    for x0 in x0_list:
        traj = newton_method_nd(f, grad_f, hess_f, x0, tol=tol)
        results[tuple(x0)] = traj

    # 4. print results
    print("Gradient:", grad_expr)
    print("Hessian :")
    print(hess_expr)

    for start, traj in results.items():
        print(f"x0={start} -> x*={traj[-1]}, iters={len(traj)-1}")

    # 5. plot
    if show_plot:
        plot_newton_2d(f, results)


# ===============================================================
# 5️⃣ VÍ DỤ CHẠY — 2D FUNCTION
# ===============================================================
if __name__ == "__main__":

    # define variables
    x1, x2 = sp.symbols("x1 x2")
    variables = [x1, x2]

    # define f(x1, x2)
    f_expr = (x1 - 1)**2 + (x2 + 3)**2 + sp.sin(x1)*0.5

    run_newton_general_nd(
        f_expr=f_expr,
        variables=variables,
        x0_list=[
            [3, -1],
            [0, 0],
            [-2, 4]
        ],
        tol=1e-8,
        show_plot=True
    )


**Câu 2: (4 điểm).**
Xét hàm số $P = \begin{bmatrix} 1 & 0 \\ 0 & 4 \end{bmatrix}$ và vector $q = (-1; 4)$. Xét bài toán tối ưu sau
$$
\min_{x \in \mathbb{R}^2} f(x) = \frac{1}{2} x^T Px + q^T x + \frac{5}{2}. \quad (1)
$$

**a. (2 điểm)** Xác định điểm tối ưu $x^*$ và giá trị tối ưu $p^*$ của bài toán (1).

**b. (1 điểm)** Chuyển bài toán (1) về bài toán least square (bình phương tối thiểu) sau bằng cách chỉ ra ma trận $A$ và vector $b$.
$$
\min_{x \in \mathbb{R}^2} \frac{1}{2} \| Ax - b \|_2^2. \quad (2)
$$
Biết rằng $A$ là ma trận đường chéo và vector $b$ có các thành phần là các số thực dương.

**c. (1 điểm)** Sử dụng thuật toán Gradient Descent cho bài toán trên (bài toán (1) hoặc (2)), với giá trị $x$ ban đầu là $x^{(0)} = (-1; 2)$, sử dụng learning rate lần lượt là $0.4$ và $0.6$ và thực hiện tối đa $100$ vòng lặp. In ra giá trị của $x^{(k)}$, $f(x^{(k)})$ tương ứng sau mỗi vòng lặp $k$ và vẽ đồ thị biểu thị cho sai số $|f(x^{(k)}) - p^*|$ trong cả hai trường hợp của learning rate. Từ đó hãy đưa ra kết luận về sự hội tụ của thuật toán Gradient Descent trong từng trường hợp của learning rate.

In [ ]:
import numpy as np

def least_square_to_quad(A, b):
    """
    Input:
        A: number (1D) hoặc ma trận (ND)
        b: number (1D) hoặc vector (ND)

    Output:
        P, q, c such that:
        1/2 ||Ax - b||^2 = 1/2 x^T P x + q^T x + c
    """

    # ===== 1D case =====
    if np.isscalar(A):
        p = A*A
        q = -A*b
        c = 0.5*b*b
        return p, q, c

    # ===== ND case =====
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)

    P = A.T @ A
    q = -A.T @ b
    c = 0.5 * (b @ b)

    return P, q, c
A = np.array([[1, 0], [0, -2]])
b = np.array([1, 2])
p, q, c = least_square_to_quad(A, b)
print(P)
print(q)
print(c)

In [ ]:
import numpy as np

def to_least_square(P, q, positive_b=True):
    """
    Chuyển bài toán dạng 1/2 x^T P x + q^T x + c
    → dạng least square 1/2 ||A x - b||^2 + const
    
    Tham số:
    ----------
    P : ndarray (n x n)
        Ma trận đối xứng dương xác định (Hessian)
    q : ndarray (n,)
        Vector hệ số tuyến tính
    positive_b : bool
        Nếu True → ép b có các phần tử dương (bằng cách điều chỉnh dấu của A)
    
    Trả về:
    ----------
    A : ndarray (n x n)
        Ma trận A sao cho A^T A = P
    b : ndarray (n,)
        Vector b sao cho A^T b = -q
    """
    # --- 1️⃣ Lấy A sao cho A^T A = P (dùng căn ma trận chéo)
    # Vì P là chéo nên ta chỉ cần sqrt từng phần tử trên đường chéo
    diag_P = np.diag(P)
    if np.any(diag_P <= 0):
        raise ValueError("P phải là ma trận dương xác định.")
    A = np.diag(np.sqrt(diag_P))

    # --- 2️⃣ Giải A^T b = -q  →  b = -(A^{-T}) q
    b = -np.linalg.inv(A.T) @ q

    # --- 3️⃣ Nếu muốn b dương thì đổi dấu dòng tương ứng của A, b
    if positive_b:
        for i in range(len(b)):
            if b[i] < 0:
                A[i, :] *= -1
                b[i] *= -1
    return A, b

def quad_to_least_square(P, q):
    """
    Input:
        P: scalar (1D) or diagonal matrix (nD) positive definite
        q: scalar (1D) or vector (nD)
    
    Output:
        A, b such that:
        1/2 x^T P x + q^T x + c = 1/2 ||Ax - b||^2 + constant
        where:
            A diagonal
            b > 0
    """

    # ============ CASE 1: ONE DIMENSION =============
    if np.isscalar(P):
        p = P
        qv = q
    
        # p must > 0
        if p <= 0:
            raise ValueError("P must be positive in 1D")

        a = np.sqrt(p)
        
        # force b positive
        b = abs(qv) / np.sqrt(p)
        
        # adjust sign of a to satisfy -ab = q
        if qv > 0:
            a = -a   # give q positive
        
        return a, b

    # ============ CASE ND: MATRIX INPUT =============
    P = np.array(P, dtype=float)
    q = np.array(q, dtype=float)

    n = len(q)

    # check diagonal
    if not np.allclose(P, np.diag(np.diag(P))):
        raise ValueError("P must be diagonal for this construction")

    A = np.zeros_like(P)
    b = np.zeros(n)

    for i in range(n):
        p_i = P[i, i]
        q_i = q[i]

        if p_i <= 0:
            raise ValueError("P must be positive diagonal matrix")

        # construct b positive
        b[i] = abs(q_i) / np.sqrt(p_i)

        # assign sign to satisfy -a_i b_i = q_i
        sign = -np.sign(q_i) if q_i != 0 else -1

        A[i, i] = sign * np.sqrt(p_i)

    return A, b
def least_square_plus_to_quad(A, b):
    """
    Convert: 1/2 ||Ax + b||^2 → quadratic form
    Output: P, q, c  such that:
    1/2 x^T P x + q^T x + c
    """

    # ========== CASE 1D ==========
    if np.isscalar(A):

        p = A*A
        q = A*b
        c = 0.5*b*b

        return p, q, c

    # ========== CASE ND ==========
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)

    P = A.T @ A
    q = A.T @ b
    c = 0.5 * (b @ b)

    return P, q, c

# ===============================
# 🔹 ÁP DỤNG CHO CÂU (b)
# ===============================
P = np.array([[1.0, 0.0],
              [0.0, 4.0]])
q = np.array([-1.0, 4.0])

A, b = quad_to_least_square(P, q)

print("✅ Ma trận A:")
print(A)
print("\n✅ Vector b:")
print(b)

# kiểm tra lại điều kiện
print("\nKiểm tra lại:")
print("A^T A =\n", A.T @ A)
print("A^T b = ", A.T @ b, "  (phải bằng -q =", -q, ")")


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sympy as sp
from scipy.linalg import inv
from scipy.optimize import minimize
from tabulate import tabulate


# ============================================================
# 🧩 1️⃣ Phần a — Giải nghiệm tối ưu phân tích & bằng thư viện
# ============================================================
def solve_quadratic_analytic(P, q, c=0):
    """
    Giải nghiệm tối ưu của f(x) = 1/2 x^T P x + q^T x + c
    Bằng công thức giải tích: x* = -P^{-1} q
    """
    x_star = -inv(P) @ q
    f_star = 0.5 * x_star.T @ P @ x_star + q.T @ x_star + c
    return x_star, f_star


def solve_quadratic_scipy(P, q, c=0):
    """
    Giải bằng scipy.optimize.minimize để kiểm chứng
    """
    def f(x):
        return 0.5 * x @ P @ x + q @ x + c

    res = minimize(f, x0=np.zeros_like(q))
    return res.x, res.fun
P = np.array([[1, 0], [0, 4]], dtype=float)
q = np.array([-1, 4], dtype=float)
c = 2.5
x0 = np.array([-1.0, 2.0])

# --- (a) Giải analytic & bằng SciPy ---
x_star, f_star = solve_quadratic_analytic(P, q, c)
x_scipy, f_scipy = solve_quadratic_scipy(P, q, c)
print("=== (a) Kết quả ===")
print("x* (analytic) =", x_star, "  p* =", f_star)
print("x* (scipy)    =", x_scipy, "  f* =", f_scipy, "\n")
import numpy as np

def to_least_square_form(P, q):
    """
    Tìm A, b sao cho:
        1/2 ||A x - b||^2 = 1/2 x^T P x + q^T x + const

    Giả sử:
        - P là ma trận đường chéo dương (PSD + diag)
        - A là ma trận đường chéo
        - b_i > 0 với mọi i
    """

    P = np.array(P, dtype=float)
    q = np.array(q, dtype=float)

    # kiểm tra P là đường chéo
    if not np.allclose(P, np.diag(np.diag(P))):
        raise ValueError("P phải là ma trận đường chéo cho hàm này.")

    diagP = np.diag(P)
    if np.any(diagP <= 0):
        raise ValueError("P phải có đường chéo dương (positive diagonal).")

    n = len(diagP)
    A = np.zeros_like(P)
    b = np.zeros(n)

    for i in range(n):
        p_i = diagP[i]
        q_i = q[i]

        # |a_i| = sqrt(p_i)
        base = np.sqrt(p_i)

        # chọn b_i > 0
        # ta muốn: -a_i * b_i = q_i
        # => b_i = -q_i / a_i
        # cho b_i > 0 ⇒ sign(a_i) = -sign(q_i)
        # nếu q_i = 0, ta có thể chọn bất kỳ, cứ lấy sign = -1 cho dễ
        if q_i == 0:
            sign_ai = -1.0
            b_i = 0.0  # b_i = 0 vẫn không âm
        else:
            sign_ai = -np.sign(q_i)
            b_i = abs(q_i) / base

        A[i, i] = sign_ai * base
        b[i] = b_i

    return A, b

A, b = to_least_square_form(P, q)
print("=== (b) Dạng Least Square ===")
print("A =\n", A)
print("b =", b, "\n")
# 🧩 3️⃣ Phần c — Gradient Descent có log bằng pandas
# ============================================================
def gradient_descent(f, grad, x0, lr=0.4, tol=1e-8, max_iter=100, f_star=None):
    """
    Gradient Descent tổng quát, log kết quả bằng DataFrame
    """
    x = x0.copy()
    history = []

    for k in range(max_iter):
        grad_k = grad(x)
        x_new = x - lr * grad_k
        f_val = f(x_new)
        err = abs(f_val - f_star) if f_star is not None else np.nan
        history.append([k, *x_new, f_val, err])

        if np.linalg.norm(x_new - x) < tol:
            break
        x = x_new

    df = pd.DataFrame(history, columns=['iter', 'x1', 'x2', 'f(x)', '|f - f*|'])
    return df


# ============================================================
# 🧩 4️⃣ Vẽ đồ thị sai số bằng seaborn (log-scale)
# ============================================================
def plot_convergence(df_list, lr_list):
    sns.set(style="whitegrid", font_scale=1.2)
    plt.figure(figsize=(8, 5))
    for df, lr in zip(df_list, lr_list):
        sns.lineplot(x='iter', y='|f - f*|', data=df, label=f'lr={lr}')
    plt.yscale('log')
    plt.title('Hội tụ Gradient Descent (log scale)')
    plt.xlabel('Iteration')
    plt.ylabel('|f(x) - f*|')
    plt.legend()
    plt.show()
# --- (c) Gradient Descent ---
# Tạo f và grad dùng sympy (cho tự động)
x1, x2 = sp.symbols('x1 x2')
f_expr = 0.5 * (x1**2 + 4*x2**2) - x1 + 4*x2 + 2.5
grad_syms = [sp.diff(f_expr, v) for v in (x1, x2)]
f_np = sp.lambdify((x1, x2), f_expr, 'numpy')
grad_np = lambda x: np.array([float(g.subs({x1:x[0], x2:x[1]})) for g in grad_syms])

f = lambda x: f_np(x[0], x[1])
grad = grad_np

# Hai learning rates
df_04 = gradient_descent(f, grad, x0, lr=0.4, f_star=f_star)
df_06 = gradient_descent(f, grad, x0, lr=0.6, f_star=f_star)

print("=== (c) Bảng log (lr=0.4) ===")
print(tabulate(df_04.head(10), headers='keys', tablefmt='psql', floatfmt=".6f"))
print("\n=== (c) Bảng log (lr=0.6) ===")
print(tabulate(df_06.head(10), headers='keys', tablefmt='psql', floatfmt=".6f"))

# Vẽ đồ thị sai số
plot_convergence([df_04, df_06], [0.4, 0.6])

# --- Nhận xét ---
print("\n=== Kết luận ===")
print("• Với lr=0.4 < 0.5 → hội tụ ổn định (sai số giảm đều).")
print("• Với lr=0.6 > 0.5 → sai số dao động hoặc tăng, thuật toán không hội tụ.")

**Câu 3: (4 điểm).**
Cho ma trận $U = \begin{bmatrix} 1 & -2 \\ 2 & -4 \end{bmatrix}$ và vector $w = \begin{pmatrix} 2 \\ 2 \end{pmatrix}$. Xét bài toán tối ưu có điều kiện ràng buộc của bài toán (1) như sau (ma trận $P$ và vector $q$ được cho ở Câu 1)
$$
\min_{x=(x_1, x_2) \in \mathbb{R}^2} f(x) = \frac{1}{2} x^T Px + q^T x + \frac{5}{2},
$$
$$
\text{subject to} \quad Ux - w = 0.
$$

**a. (2 điểm)** Gọi $x^*$ và $v^*$ lần lượt là điểm tối ưu của bài toán trên và bài toán đối ngẫu của nó, viết **điều kiện Karush-Kuhn-Tucker (KKT)** cho $x^*$ và $v^*$.

**b. (2 điểm)** Tìm một cặp giá trị của $(x^*, v^*)$ thỏa điều kiện KKT vừa tìm được ở câu b), từ đó đưa ra giá trị tối ưu cho bài toán trên.

# a

In [ ]:
import sympy as sp

def KKT_conditions(P, q, U=None, w=None):
    """
    Sinh điều kiện KKT cho bài toán tổng quát:

        minimize 1/2 x^T P x + q^T x + constant
        subject to Ux - w = 0     (equality constraints)

    Input:
        P : n×n sympy matrix
        q : n vector
        U : m×n matrix (optional)
        w : m vector  (optional)

    Output:
        In ra Stationarity và Feasibility conditions
        
    -----------------------------------------------------------
    Nếu không có U,w → chỉ in stationarity cho unconstrained.
    -----------------------------------------------------------
    """

    # =======================
    # Convert to sympy objects
    # =======================
    P = sp.Matrix(P)
    q = sp.Matrix(q)

    n = P.shape[0]                   # number of x variables
    x = sp.Matrix(sp.symbols(f'x1:{n+1}', real=True))
    
    # Objective function
    f = (sp.Rational(1,2) * (x.T * P * x)[0] + (q.T * x)[0])

    # ==== If no constraint ====
    if U is None or w is None:
        print("\n=== STATIONARITY (unconstrained) ===")
        grad = [sp.diff(f, x[i]) for i in range(n)]
        for g in grad:
            print(f"{g} = 0")
        return

    # ==========================
    # Constrained case: add v
    # ==========================
    U = sp.Matrix(U)
    w = sp.Matrix(w)

    m = U.shape[0]                   # number of constraints
    v = sp.Matrix(sp.symbols(f'v1:{m+1}', real=True))

    # Build Lagrangian
    L = f + (v.T * (U * x - w))[0]

    # ======================
    # STATIONARITY
    # ======================
    grad_x = [sp.diff(L, x[i]) for i in range(n)]

    # ======================
    # FEASIBILITY
    # ======================
    feasibility = list(U * x - w)

    # ======================
    # PRINT RESULTS
    # ======================
    print("\n===== LAGRANGIAN =====")
    print("L = ", L)

    print("\n===== STATIONARITY CONDITIONS =====")
    for i, g in enumerate(grad_x):
        print(f"∂L/∂x{i+1}: {g} = 0")

    print("\n===== PRIMAL FEASIBILITY CONDITIONS =====")
    for i, c in enumerate(feasibility):
        print(f"constraint {i+1}: {c} = 0")

    print("\n===== DUAL FEASIBILITY =====")
    print("v free (no sign constraints)")

    print("\n===== COMPLEMENTARY SLACKNESS =====")
    print("N/A because only equalities exist.")
P = [[1,0],[0,4]]
q = [-1,4]
U = [[1,-2],[2,-4]]
w = [2,2]

KKT_conditions(P,q,U,w)


In [ ]:
import sympy as sp

def KKT_general(f_expr, x_symbols, 
                eq_constraints=None, 
                ineq_constraints=None):

    if eq_constraints is None:
        eq_constraints = []
    if ineq_constraints is None:
        ineq_constraints = []

    x = sp.Matrix(x_symbols)
    n = len(x_symbols)

    # -----------------------------
    # Lagrange multipliers
    # -----------------------------
    if eq_constraints:
        v = sp.Matrix(sp.symbols(f"v1:{len(eq_constraints)+1}", real=True))
    else:
        v = sp.Matrix([])

    if ineq_constraints:
        lam = sp.Matrix(sp.symbols(f"l1:{len(ineq_constraints)+1}", real=True))
    else:
        lam = sp.Matrix([])

    # -----------------------------
    # Build Lagrangian
    # -----------------------------
    L = f_expr

    for i, h in enumerate(eq_constraints):
        L += v[i] * h

    for j, g in enumerate(ineq_constraints):
        L += lam[j] * g

    # -----------------------------
    # Stationarity
    # -----------------------------
    stationarity = [sp.simplify(sp.diff(L, x[i])) for i in range(n)]

    # -----------------------------
    # Feasibility
    # -----------------------------
    eq_feas = [sp.simplify(h) for h in eq_constraints]
    ineq_feas = [sp.simplify(g) for g in ineq_constraints]

    # -----------------------------
    # Print out KKT
    # -----------------------------
    print("\n===== KKT CONDITIONS =====")

    print("\n--- STATIONARITY ---")
    for s in stationarity:
        print(f"{s} = 0")

    if eq_constraints:
        print("\n--- EQUALITY FEASIBILITY ---")
        for h in eq_feas:
            print(f"{h} = 0")

    if ineq_constraints:
        print("\n--- INEQUALITY FEASIBILITY ---")
        for g in ineq_feas:
            print(f"{g} <= 0")

        print("\n--- DUAL FEASIBILITY ---")
        for j in range(len(lam)):
            print(f"l{j+1} >= 0")

        print("\n--- COMPLEMENTARY SLACKNESS ---")
        for j, g in enumerate(ineq_feas):
            print(f"l{j+1} * ({g}) = 0")
x1, x2 = sp.symbols('x1 x2', real=True)
f_expr = 0.5*(x1**2 + 4*x2**2) - x1 + 4*x2 + 2.5


In [ ]:
import sympy as sp

def KKT_general1(f_expr, x_symbols, 
                eq_constraints=None, 
                ineq_constraints=None,
                show_lagrangian=True):

    if eq_constraints is None:
        eq_constraints = []
    if ineq_constraints is None:
        ineq_constraints = []

    x = sp.Matrix(x_symbols)
    n = len(x_symbols)

    # -----------------------------
    # DEFINE MULTIPLIERS
    # -----------------------------
    if eq_constraints:
        v = sp.Matrix(sp.symbols(f"v1:{len(eq_constraints)+1}", real=True))
    else:
        v = sp.Matrix([])

    if ineq_constraints:
        lam = sp.Matrix(sp.symbols(f"l1:{len(ineq_constraints)+1}", real=True))
    else:
        lam = sp.Matrix([])

    # -----------------------------
    # BUILD LAGRANGIAN
    # -----------------------------
    L = f_expr

    for i, h in enumerate(eq_constraints):
        L += v[i] * h

    for j, g in enumerate(ineq_constraints):
        L += lam[j] * g

    # -----------------------------
    # PRINT LAGRANGIAN
    # -----------------------------
    if show_lagrangian:
        print("\n===== LAGRANGIAN =====")
        print("L(x, multipliers) =")
        print(sp.simplify(L))

    # -----------------------------
    # STATIONARITY ∂L/∂x = 0
    # -----------------------------
    stationarity = [sp.diff(L, x[i]) for i in range(n)]

    print("\n===== STATIONARITY CONDITIONS =====")
    for i, s in enumerate(stationarity):
        print(f"∂L/∂x{i+1} = {sp.simplify(s)} = 0")

    # -----------------------------
    # PRIMAL FEASIBILITY
    # -----------------------------
    if eq_constraints:
        print("\n===== EQUALITY FEASIBILITY =====")
        for h in eq_constraints:
            print(f"{sp.simplify(h)} = 0")

    if ineq_constraints:
        print("\n===== INEQUALITY FEASIBILITY =====")
        for g in ineq_constraints:
            print(f"{sp.simplify(g)} <= 0")

        # dual feasibility
        print("\n===== DUAL FEASIBILITY =====")
        for j in range(len(lam)):
            print(f"λ{j+1} >= 0")

        # slackness
        print("\n===== COMPLEMENTARY SLACKNESS =====")
        for j, g in enumerate(ineq_constraints):
            print(f"λ{j+1} * ({sp.simplify(g)}) = 0")
x1, x2 = sp.symbols('x1 x2', real=True)
f_expr = 0.5*(x1**2 + 4*x2**2) - x1 + 4*x2 + 2.5


In [ ]:
U = sp.Matrix([[1,-2],
               [2,-4]])

w = sp.Matrix([2,2])

xvec = sp.Matrix([x1,x2])

eq = list(U*xvec - w)

KKT_general1(f_expr, [x1,x2], eq_constraints=eq)


In [ ]:
# rang buoc  x1 + x2 -3 = 0
eq = [x1 + x2 - 3]
KKT_general1(f_expr, [x1,x2], eq_constraints=eq)


In [ ]:
# rang buoc x1 -2 <= 0
ineq = [x1 - 2]
KKT_general(f_expr, [x1,x2], ineq_constraints=ineq)


In [ ]:
# rang buoc hon hop x1 + x2 = 0
#                   x1 - 5 <= 0
#                   -x2 -1 <= 0
eq = [x1 + x2]
ineq = [x1 - 5, -x2 - 1]

KKT_general(f_expr, [x1,x2], 
            eq_constraints=eq, 
            ineq_constraints=ineq)


# B

In [ ]:
import sympy as sp

# =========================
# 1. Khai báo biến & dữ liệu
# =========================

x1, x2 = sp.symbols('x1 x2', real=True)
v1, v2 = sp.symbols('v1 v2', real=True)

# P, q từ Câu 1
P = sp.Matrix([[1, 0],
               [0, 4]])
q = sp.Matrix([-1, 4])

# U, w từ đề Câu 3
U = sp.Matrix([[1, -2],
               [2, -4]])
w = sp.Matrix([2, 2])

x = sp.Matrix([x1, x2])
v = sp.Matrix([v1, v2])

# =========================
# 2. Hàm mục tiêu & Lagrangian
# =========================

f_expr = 0.5 * (x.T * P * x)[0] + (q.T * x)[0] + sp.Rational(5, 2)
L = f_expr + (v.T * (U*x - w))[0]

print("Lagrangian L(x,v) =")
print(sp.simplify(L))

# =========================
# 3. Điều kiện KKT (stationarity + feasibility)
# =========================

# ∂L/∂x = 0
eq_station = [
    sp.Eq(sp.diff(L, x1), 0),
    sp.Eq(sp.diff(L, x2), 0)
]

# Ux - w = 0
Ux_minus_w = U*x - w
eq_feas = [
    sp.Eq(Ux_minus_w[0], 0),
    sp.Eq(Ux_minus_w[1], 0)
]

all_eqs = eq_station + eq_feas

print("\n===== HỆ PHƯƠNG TRÌNH KKT =====")
for e in all_eqs:
    print(e)

# =========================
# 4. Giải hệ phương trình KKT
# =========================

sol = sp.solve(all_eqs, (x1, x2, v1, v2), dict=True)
print("\n===== NGHIỆM KKT =====")
print(sol)

if sol:
    sol0 = sol[0]
    x_star = sp.Matrix([sol0[x1], sol0[x2]])
    v_star = sp.Matrix([sol0[v1], sol0[v2]])
    p_star = sp.simplify(f_expr.subs(sol0))

    print("\nMột cặp (x*, v*) thỏa KKT:")
    print("x* =", x_star)
    print("v* =", v_star)
    print("Giá trị tối ưu f(x*) =", p_star)
else:
    print("\n❌ Không tồn tại (x*, v*) thỏa hệ KKT → bài toán ràng buộc Ux - w = 0 là vô nghiệm (primal infeasible).")


# b

In [ ]:
import numpy as np

def solve_kkt_qp(P, q, U, w, c=0.0, tol=1e-10):
    """
    Giải bài toán:
        minimize   1/2 x^T P x + q^T x + c
        subject to U x = w

    - Tự phát hiện U bị phụ thuộc hàng (rank thấp) và rút gọn.
    - Trả về (x*, v*, f*)

    Tham số
    --------
    P : (n, n) ndarray
    q : (n,) ndarray
    U : (m, n) ndarray
    w : (m,) ndarray
    c : số thực, hằng số trong hàm mục tiêu (ở đề là 5/2)
    tol : ngưỡng để nhận biết rank thấp

    Trả về
    --------
    x_star : (n,) nghiệm tối ưu
    v_star : (r,) bội Lagrange (r = rank của U)
    f_star : giá trị tối ưu
    """
    P = np.atleast_2d(P)
    q = np.atleast_1d(q)
    U = np.atleast_2d(U)
    w = np.atleast_1d(w)

    n = P.shape[0]

    # --- 1) phát hiện rank của U ---
    # dùng SVD để xem hàng nào độc lập
    Uu, s, Vt = np.linalg.svd(U)
    rank = np.sum(s > tol)

    if rank < U.shape[0]:
        # chỉ giữ lại đúng số hàng độc lập
        U_red = U[:rank, :]
        w_red = w[:rank]
    else:
        U_red, w_red = U, w

    m = U_red.shape[0]

    # --- 2) lập hệ KKT ---
    # [ P     U^T ] [x] = [-q]
    # [ U      0  ] [v]   [ w]
    KKT = np.block([
        [P, U_red.T],
        [U_red, np.zeros((m, m))]
    ])
    rhs = np.concatenate([-q, w_red])

    # --- 3) giải hệ ---
    try:
        sol = np.linalg.solve(KKT, rhs)
    except np.linalg.LinAlgError:
        # phòng trường hợp vẫn singular
        sol = np.linalg.pinv(KKT) @ rhs

    x_star = sol[:n]
    v_star = sol[n:]

    # --- 4) tính giá trị tối ưu ---
    f_star = 0.5 * x_star @ (P @ x_star) + q @ x_star + c

    return x_star, v_star, f_star


if __name__ == "__main__":
    # DỮ LIỆU ĐỀ
    P = np.array([[1., 0.],
                  [0., 4.]])
    q = np.array([-1., 4.])
    U = np.array([[1., -2.],
                  [2., -4.]])
    w = np.array([2., 2.])
    c = 2.5   # 5/2 trong đề

    x_star, v_star, f_star = solve_kkt_qp(P, q, U, w, c)

    print("✅ Nghiệm tối ưu (x*):")
    print(x_star)
    print("\n✅ Bội Lagrange (v*):")
    print(v_star)
    print("\n✅ Giá trị tối ưu f(x*):")
    print(f_star)

    # kiểm tra lại KKT để chắc ăn
    # (dùng U đã rút gọn để kiểm tra)
    # rút gọn lại như trong hàm
    Uu, s, Vt = np.linalg.svd(U)
    rank = np.sum(s > 1e-10)
    if rank < U.shape[0]:
        U_red = U[:rank, :]
        w_red = w[:rank]
    else:
        U_red, w_red = U, w

    grad_L = P @ x_star + q + U_red.T @ v_star
    feas = U_red @ x_star - w_red

    print("\n🔎 Kiểm tra KKT:")
    print("P x + q + U^T v =", grad_L)
    print("U x - w =", feas)


In [ ]:
import numpy as np

def solve_kkt_qp(P, q, U, w, c=0.0, tol=1e-10, check_kkt=True):
    """
    Giải bài toán:
        minimize   1/2 x^T P x + q^T x + c
        subject to U x = w
    """
    P = np.atleast_2d(P)
    q = np.atleast_1d(q)
    U = np.atleast_2d(U)
    w = np.atleast_1d(w)

    n = P.shape[0]

    # --- 1️⃣ Phát hiện rank của U ---
    Uu, s, Vt = np.linalg.svd(U)
    rank = np.sum(s > tol)
    is_dependent = rank < U.shape[0]

    if is_dependent:
        print(f"⚠️ U có rank thấp ({rank}/{U.shape[0]}), tự động rút gọn hàng độc lập.")
        U_eff = U[:rank, :]
        w_eff = w[:rank]
    else:
        U_eff, w_eff = U, w

    m = U_eff.shape[0]

    # --- 2️⃣ Lập hệ KKT ---
    KKT = np.block([
        [P, U_eff.T],
        [U_eff, np.zeros((m, m))]
    ])
    rhs = np.concatenate([-q, w_eff])

    # --- 3️⃣ Giải hệ ---
    try:
        sol = np.linalg.solve(KKT, rhs)
    except np.linalg.LinAlgError:
        print("⚠️ Ma trận KKT suy biến, dùng pseudo-inverse.")
        sol = np.linalg.pinv(KKT) @ rhs

    x_star = sol[:n]
    v_star = sol[n:]

    # --- 4️⃣ Tính giá trị tối ưu ---
    f_star = 0.5 * x_star @ (P @ x_star) + q @ x_star + c

    # --- 5️⃣ Kiểm tra lại KKT (tổng quát) ---
    if check_kkt:
        grad_L = P @ x_star + q + U.T @ np.pad(v_star, (0, U.shape[0] - m))
        feas = U @ x_star - w

        print("\n🔎 KIỂM TRA ĐIỀU KIỆN KKT:")
        print(f"Px + q + Uᵀv = {grad_L}")
        print(f"Ux - w = {feas}")

        if np.allclose(grad_L, 0, atol=1e-6) and np.allclose(feas, 0, atol=1e-6):
            print("✅ Thỏa điều kiện KKT.")
        else:
            print("⚠️ Không hoàn toàn thỏa KKT (cần xem lại ràng buộc hoặc rank của U).")

    return x_star, v_star, f_star


# ===============================
# 🎯 DEMO: DỮ LIỆU ĐỀ
# ===============================
P = np.array([[1., 0.],
              [0., 4.]])
q = np.array([-1., 4.])
U = np.array([[1., -2.],
              [2., -4.]])
w = np.array([2., 2.])
c = 2.5

x_star, v_star, f_star = solve_kkt_qp(P, q, U, w, c)

print("\n✅ Nghiệm tối ưu:")
print("x* =", x_star)
print("v* =", v_star)
print("f(x*) =", f_star)


In [ ]:
import numpy as np
from scipy.linalg import solve

def remove_dependent_rows(U, w, tol=1e-10):
    """
    Loại bỏ các hàng phụ thuộc tuyến tính trong U (và phần tử tương ứng trong w).
    """
    # SVD decomposition
    Uu, s, Vt = np.linalg.svd(U)
    rank = np.sum(s > tol)
    if rank < U.shape[0]:
        print(f"⚠️ Ma trận U có rank thấp ({rank}/{U.shape[0]}), sẽ tự động rút gọn.")
        # Giữ lại các hàng độc lập tuyến tính
        U_reduced = U[:rank, :]
        w_reduced = w[:rank]
        return U_reduced, w_reduced
    return U, w


def solve_kkt(P, q, U, w):
    """
    Giải hệ KKT cho bài toán:
        min (1/2)x^T P x + q^T x
        subject to Ux = w
    """
    # Loại bỏ hàng phụ thuộc nếu có
    U, w = remove_dependent_rows(U, w)

    n = P.shape[0]
    m = U.shape[0]

    # --- Ma trận KKT ---
    K = np.block([
        [P, U.T],
        [U, np.zeros((m, m))]
    ])
    b = np.concatenate([-q, w])

    # --- Giải hệ ---
    sol = solve(K, b, assume_a='sym')
    x_star = sol[:n]
    v_star = sol[n:]
    return x_star, v_star


def quadratic_value(P, q, c, x):
    """Tính f(x) = 1/2 x^T P x + q^T x + c"""
    return 0.5 * x.T @ P @ x + q.T @ x + c


# ===============================
# 🔹 DỮ LIỆU BÀI TOÁN CỤ THỂ
# ===============================
P = np.array([[1.0, 0.0],
              [0.0, 4.0]])
q = np.array([-1.0, 4.0])
c = 2.5

U = np.array([[1, -2],
              [2, -4]])
w = np.array([2, 2])

# ===============================
# 🔹 GIẢI HỆ KKT
# ===============================
x_star, v_star = solve_kkt(P, q, U, w)

print("\n✅ Nghiệm tối ưu:")
print("x* =", x_star)
print("v* =", v_star)

f_star = quadratic_value(P, q, c, x_star)
print("\n✅ Giá trị tối ưu f(x*) =", f_star)


In [ ]:
import numpy as np

def solve_quadratic_eq(P, q, U=None, w=None):
    """
    Giải bài toán QP dạng:
        minimize    0.5 x^T P x + q^T x + c   (c không ảnh hưởng nghiệm)
        subject to  U x = w

    Tham số
    --------
    P : (n, n) ndarray
        Ma trận đối xứng xác định dương (hoặc nửa xác định dương)
    q : (n,) ndarray
        Vector tuyến tính
    U : (m, n) ndarray hoặc None
        Ma trận ràng buộc đẳng thức. Nếu None -> bài toán không ràng buộc.
    w : (m,) ndarray hoặc None
        Vế phải của ràng buộc. Phải cùng số dòng với U.

    Trả về
    -------
    x_star : (n,) ndarray
        Nghiệm tối ưu tìm được
    v_star : (m,) ndarray hoặc None
        Bội số Lagrange của ràng buộc. Nếu không có ràng buộc -> None
    """
    P = np.atleast_2d(P)
    q = np.atleast_1d(q)

    n = P.shape[0]

    # Không có ràng buộc: nghiệm = -P^{-1} q
    if U is None or w is None:
        x_star = -np.linalg.solve(P, q)
        return x_star, None

    U = np.atleast_2d(U)
    w = np.atleast_1d(w)
    m = U.shape[0]

    # Xây hệ KKT:
    # [ P   U^T ] [x] = [-q]
    # [ U    0 ] [v]   [ w]
    KKT = np.block([
        [P, U.T],
        [U, np.zeros((m, m))]
    ])
    rhs = np.concatenate([-q, w])

    # Vì KKT có thể singular (như đề này) -> dùng pinv để có 1 nghiệm
    sol = np.linalg.pinv(KKT) @ rhs

    x_star = sol[:n]
    v_star = sol[n:]

    return x_star, v_star


if __name__ == "__main__":
    # DỮ LIỆU ĐỀ BÀI
    P = np.array([[1., 0.],
                  [0., 4.]])
    q = np.array([-1., 4.])
    U = np.array([[1., -2.],
                  [2., -4.]])
    w = np.array([2., 2.])

    x_star, v_star = solve_quadratic_eq(P, q, U, w)

    print("x* tìm được =")
    print(x_star)
    print("v* (bội Lagrange) =")
    print(v_star)

    # kiểm tra lại điều kiện KKT
    grad_L = P @ x_star + q + U.T @ v_star
    feas = U @ x_star - w

    print("\nKiểm tra KKT:")
    print("P x + q + U^T v = ", grad_L)
    print("U x - w = ", feas)

    # Giá trị tối ưu
    f_opt = 0.5 * x_star @ (P @ x_star) + q @ x_star + 2.5  # +5/2 theo đề
    print("\nGiá trị tối ưu f(x*) =", f_opt)
